## Load Data

In [ ]:
import os, sys, time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
plt.rcParams['figure.dpi'] = 240 # fix high-dpi display scaling issues (only if you have a 4k monitor)

sys.path.append(os.getcwd()) # add cwd to path

from zip_codes import ZC # zip code database
import load_file as lf # file i/o
import myplots as mp # my plotting functions
import file_reshaper as fr # file reshaper

zc = ZC(fdir='') # initialize zip code class

In [ ]:
data_dir = r'C:\PythonBC\RootData'

fname = 'clicks.gzip'
df_clicks = lf.temp_load( os.path.join(data_dir, fname)  )

fname = 'state.gzip'
df_state = lf.temp_load( os.path.join(data_dir, fname)  )

fname = 'geo_zip.gzip'
df_geozip = lf.temp_load( os.path.join(data_dir, fname)  )

frames = [df_state, df_geozip, df_clicks]
df = pd.concat(frames, axis=1)

## Make a folium map

In [ ]:
import folium
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
#state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
#state_data = pd.read_csv(state_unemployment)


In [ ]:
dfstates = df.groupby('state').sum()['clicks'].to_frame()
dfstates.reset_index(level=0, inplace=True)

#m = folium.Map(location=[48, -102], zoom_start=3) # all 50 states
m = folium.Map(location=[39.50, -98.35], zoom_start=4) # lower 48
folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=dfstates,
    columns=['state', 'clicks'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='clicks'
).add_to(m)
folium.LayerControl().add_to(m)
m

## Zip code level map of an individual state

In [ ]:
df.state.value_counts()

In [ ]:
df_state = df[df.state == 'ND']
df_state.geo_zip = df_state.geo_zip.apply(int)
print(df_state.shape)
dfzips = df.groupby('geo_zip').sum()['clicks'].to_frame()
dfzips.reset_index(level=0, inplace=True)

In [ ]:
import folium
json_dir = r'C:\PythonBC\State-zip-code-GeoJSON'
fname_json = 'map_ND.geojson'
zip_geo = os.path.join(json_dir, fname_json)
print(zip_geo)
m = folium.Map(location=[47.5515, -101.0020], zoom_start=6) # north dakota
folium.Choropleth(
    geo_data=zip_geo,
    name='choropleth',
    data=dfzips,
    columns=['geo_zip', 'clicks'],
    key_on="feature.properties.ZCTA5CE10",
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='clicks'
).add_to(m)
folium.LayerControl().add_to(m)
m
m.save('index.html')


In [ ]:
print(df_state.geo_zip.iloc[0])
print(type(df_state.geo_zip.iloc[0]))

In [ ]:
type(dfzips)